# Introducción
A continuación detalla el plan de trabajo para el análisis de datos del servicio de telefonía virtual CallMeMaybe. El objetivo principal es identificar a los operadores menos eficaces basándose en métricas clave de rendimiento. Esta iniciativa busca proporcionar a los supervisores información valiosa para mejorar la eficiencia operativa y la calidad del servicio.

# Objetivo del proyecto
El objetivo principal de este proyecto es identificar a los operadores menos eficientes utilizando datos históricos de llamadas. Se considerará a un operador ineficaz si cumple con uno o más de los siguientes criterios:
- Alta cantidad de llamadas entrantes perdidas (internas y externas).
- Tiempo de espera prolongado para las llamadas entrantes.
- Bajo número de llamadas salientes externas.

Además, para entender el contexto de la ineficacia, se analizará:
- La baja cantidad de llamadas internas y externas (un indicador de baja actividad general, si el rol del operador lo requiere).
- La cantidad de llamadas entrantes de clientes dependiendo de su tipo de plan (tariff_plan) y su fecha de ingreso al plan (date_start). 

Esto ayudará a contextualizar la carga de trabajo y el rendimiento del operador.

# Descripción de los Datos
Se utilizarán dos datasets principales para este análisis:

1. telecom_dataset_us.csv
Este dataset contiene la información detallada de las llamadas realizadas a través del servicio CallMeMaybe.
    - **user_id:** ID de la cuenta del cliente.
    - **date:** Fecha en que se recuperaron las estadísticas.
    - **direction:** Dirección de la llamada ("out" para saliente, "in" para entrante).
    - **internal:** Indica si la llamada fue interna (entre operadores del mismo cliente).
    - **operator_id:** Identificador único del operador.
    - **is_missed_call:** Booleano que indica si fue una llamada perdida.
    - **calls_count:** Número de llamadas.
    - **call_duration:** Duración de la llamada (sin incluir el tiempo de espera).
    - **total_call_duration:** Duración total de la llamada (incluido el tiempo de espera).

2. telecom_clients_us.csv
Este dataset proporciona información sobre los clientes de CallMeMaybe.
    - **user_id:** ID de usuario/a.
    - **tariff_plan:** Tarifa actual del cliente.
    - **date_start:** Fecha de registro del cliente.

# Métricas clave para la Ineficiencia
Para identificar a los operadores ineficaces, se definirán las siguientes métricas, que serán calculadas para cada operador:
- Cantidad de llamadas entrantes perdidas: Suma de **calls_count** donde **direction** es *"in"* y **is_missed_call** es *True*. Esto incluirá llamadas internas y externas perdidas.
- Tiempo promedio de espera para llamadas entrantes: Se calculará como **total_call_duration - call_duration** para llamadas entrantes (direction es *"in"*). Se determinará el promedio de este valor por operador.
- Cantidad de llamadas salientes: Suma de **calls_count** donde direction es *"out"*. Se analizará esto en relación con las expectativas del rol del operador.
- Cantidad de llamadas salientes externas: Suma de **calls_count** donde direction es *"out"* e internal es *False*. Una baja cantidad será un signo de ineficacia.
- Contexto de llamadas entrantes por cliente: Se analizará la cantidad de llamadas entrantes de clientes, considerando su **tariff_plan** y **date_start**, para entender si hay un impacto en el volumen de llamadas que los operadores deben manejar.

# Fases del plan de trabajo
## Fase 1: Recopilación y Preparación de Datos
- Cargar los datasets: Importar telecom_dataset_us.csv y telecom_clients_us.csv en un entorno de análisis (Python/Pandas).

- Inspeccionar los datos:
    - Revisar las primeras filas de cada dataframe.
    - Verificar los tipos de datos de cada columna.
    - Identificar valores nulos o faltantes y decidir la estrategia de manejo (imputación, eliminación, etc.).

- Combinar datasets: Realizar un merge de los dos dataframes utilizando user_id para enriquecer la información de las llamadas con los detalles del cliente.

- Ingeniería de características:
    - Calcular el tiempo de espera de la llamada (total_call_duration - call_duration).
    - Convertir las columnas de fecha (date, date_start) a formato datetime para facilitar el análisis temporal.

- Eliminación de valores atípicos: 
    - Para garantizar que el análisis se base en datos representativos y evitar la distorsión por valores extremos, se eliminarán los valores atípicos. Se mantendrá solo el 95% de los datos estándar para las métricas clave como calls_count, call_duration, total_call_duration y el tiempo de espera. Esto se puede lograr utilizando métodos como el rango intercuartílico (IQR) o el percentil.

## Fase 2: Análisis Exploratorio de Datos (EDA)
- Estadísticas descriptivas:
    - Obtener estadísticas básicas (media, mediana, desviación estándar, cuartiles) para calls_count, call_duration, total_call_duration y la nueva columna de tiempo de espera.
    - Analizar la distribución de las variables numéricas.

- Análisis de distribución de llamadas:
    -   Distribución de llamadas por direction (entrantes vs. salientes).
    - Distribución de llamadas por internal (internas vs. externas).
    - Proporción de llamadas perdidas (is_missed_call).

- Análisis por operador:
    - Número de llamadas por operator_id.
    - Duración promedio de llamadas por operator_id.
    - Número de llamadas perdidas por operator_id.
    - Tiempo promedio de espera por operator_id.

- Análisis temporal:
    - Tendencias de llamadas a lo largo del tiempo (date).
    - Impacto de la fecha de inicio del cliente (date_start) y tariff_plan en el volumen de llamadas.

- Visualizaciones:
    - Histogramas y gráficos de densidad para distribuciones de variables clave.
    - Gráficos de barras para comparar métricas por operador.
    - Gráficos de series de tiempo para tendencias diarias/semanales.
    - Gráficos de dispersión para identificar posibles correlaciones entre métricas.

## Fase 3: Identificación de Operadores Ineficaces
- Cálculo de métricas por operador:
    - Agrupar los datos por operator_id y calcular las métricas definidas en la sección 4 (cantidad de llamadas salientes externas, cantidad de llamadas perdidas, tiempo promedio de espera).

- Definición de umbrales de ineficacia:
    - Establecer umbrales estadísticos o basados en el negocio para cada métrica (ej., operadores en el cuartil superior de llamadas perdidas, operadores con tiempo de espera por encima de X segundos, operadores en el cuartil inferior de llamadas salientes externas).
    - Considerar el contexto de cada operador (por ejemplo, algunos operadores solo gestionan llamadas entrantes, otros solo salientes, y algunos ambos). Esto puede requerir una segmentación previa de los operadores.

- Listado de operadores ineficaces: Generar una lista de operator_id que cumplan con los criterios de ineficacia.

## Fase 4: Pruebas de Hipótesis Estadísticas
Se formularán y probarán hipótesis para validar las observaciones del EDA y la identificación de operadores ineficaces. Ejemplos de hipótesis a probar:

- Hipótesis 1 (Llamadas Salientes Externas): ¿Existe una diferencia estadísticamente significativa en el número de llamadas salientes externas entre los operadores identificados como ineficaces y los operadores promedio?
    - Prueba: Prueba t de Student o Mann-Whitney U test (dependiendo de la normalidad de la distribución).

- Hipótesis 2 (Llamadas Perdidas): ¿El número de llamadas perdidas de los operadores ineficaces es significativamente mayor que el de los operadores eficaces?
    - Prueba: Prueba t de Student o Mann-Whitney U test.

- Hipótesis 3 (Tiempo de Espera): ¿El tiempo de espera promedio para los operadores ineficaces es significativamente mayor que el de los operadores eficaces?
    -Prueba: Prueba t de Student o Mann-Whitney U test.

- Hipótesis 4 (Impacto del Cliente): ¿Existe una correlación entre el tariff_plan o date_start del cliente y el volumen de llamadas entrantes o la tasa de llamadas perdidas de los operadores?
    - Prueba: ANOVA (para tariff_plan) o análisis de correlación (para date_start).

# Resultados Esperados
Al finalizar este proyecto, se espera obtener:
- Un análisis exploratorio de datos completo y visualizaciones que destaquen patrones y anomalías en el comportamiento de las llamadas.
- Una lista clara de operadores identificados como ineficaces, junto con las métricas que respaldan esta clasificación.
- Conclusiones de las pruebas de hipótesis estadísticas que validen o refuten las observaciones iniciales y proporcionen una base sólida para las recomendaciones.
- Un informe detallado con hallazgos, conclusiones y recomendaciones accionables para los supervisores de CallMeMaybe.

# Próximos Pasos
Una vez completado el análisis, se presentarán los hallazgos a los stakeholders por medio de los siguientes tres elementos entregables:
1. Un notebook(.ipynb)
2. Una presentación (.pdf)
3. Un archivo de texto (`dashboard.txt`). Este contendrá un enlace al dashboard de Tableau Public.